# Geolocalización

## Parámetros

In [7]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
ILE_NAME = 'ile_limpio_cluster'
GEO_NAME = 'localidades'

## Importar

### Clases

In [8]:
from mariachis.localidades import GeoLoc

ile_class = GeoLoc(BASE_DIR, ILE_NAME)
geo_class = GeoLoc(BASE_DIR, GEO_NAME, iso_country_code='MX')

### ILE limpio y con cluster

In [9]:
ile = ile_class.full_import(api=False)
ile.sample()

Archivo con nombre ile_limpio_cluster.csv fue encontrado en:
/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data
Con 79383 renglones y 73 columnas


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_p_diasgesta,rango_fsexual_vs_menarca,rango_edad_vs_menarca,rango_edad_vs_fsexual,antes_vs_despues,antes_vs_despues_detalle,cluster,nombre,conclusión general,características de minorías
23236,2017.0,ENERO,2017-01-27,SI,UNION_LIBRE,20.0,DESCONOCIDO,SECUNDARIA,TRAB_HOGAR_NO_REMUNERADX,DESCONOCIDO,...,>= 06,01 a 03,01 a 08,05 a 08,DIFERENTE,"antes: DESCONOCIDO, después: DIU",I,Madres maduras con pareja en unión libre,Trabajadorx del hogar no remuneradx de 30-35 a...,Han tenido abortos y/o cesáreas


### Localidades por URL

In [10]:
geo = geo_class.get_data()
geo.sample()

Archivo: localidades.csv.csv fue exportado exitosamente en:
/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


,country_code,postal_code,place_name,state_name,state_code,province_name,province_code,community_name,community_code,lat,lon,accuracy
131582,MX,90438,La Concepción,Tlaxcala,29,Tetla de la Solidaridad,031,,,19.4823,-98.0764,3


## Agrupación ILE

In [11]:
from numpy import percentile

ile_group = ile.assign(n=1).pivot_table(index=['entidad','alc_o_municipio'], columns='nombre', aggfunc={'n':'count'}, fill_value=0)
ile_group.columns = [x[-1] for x in ile_group.columns]
ile_group['TOTAL'] = ile_group.sum(axis=1)
ile_group['TOTAL'].describe([i/100 for i in range(90,100)])


count      674.000000
mean       117.778932
std        659.738971
min          1.000000
50%          3.000000
90%         51.000000
91%         57.430000
92%         73.160000
93%        104.000000
94%        174.860000
95%        388.250000
96%        544.080000
97%        832.920000
98%       1836.860000
99%       3118.220000
max      10057.000000
Name: TOTAL, dtype: float64

In [12]:
ile_group = ile_group[ile_group['TOTAL']>=100].copy()
aux = ile_group.copy()
for col in aux.columns:
    ile_group[col] = aux[col]/aux['TOTAL']
ile_group = ile_group.drop('TOTAL', axis=1)

In [16]:
from sklearn.cluster import KMeans

ile_group['cluster'], alc_cluster = ile_class.make_clusters(ile_group[[x for x in ile_group.columns if x!='cluster']], cluster_obj=KMeans)
ile_class.export_csv(ile_group, name_suffix='group', sep='\t', encoding='utf-16')

Archivo: ile_limpio_cluster_group.csv.csv fue exportado exitosamente en:
/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


## Municipios

### Transformación

In [ ]:
geo = geo_class.wrangling_geo(geo)
geo.sample()

### Mapa

In [ ]:
geo['boundary'].plot()

## Unión

In [ ]:
df = ile_class.merge_with_ile(ile_group, geo)
print(df.shape)
df.sample()